In [0]:
cd drive/My Drive/Dataset

/content/drive/My Drive/Dataset


In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

import matplotlib.pyplot as plt

In [0]:
# 讀取檔案 (Read Data)
data = pd.read_csv('train.csv')

# 隨機抽樣 比率為100% (Randomly Smaple data, ratio is 100%)
data = data.sample(frac = 1)

# 顯示資料的前五筆 (Show Data)
data.head()

,id,title,author,text,label
6392,6392,Politicians to Skip Boston St. Patrick’s Day P...,Liam Stack,Some of Massachusetts’s top politicians said o...,0
7160,7160,Turkish Government Arrests Opposition Parliame...,NaN,Turkish Government Arrests Opposition Parliame...,1
14155,14155,This Professor Devotes Her Life to Countering ...,Henry Wolff,"Loading Posted on November 2, 2016 This Profes...",1
833,833,Trump Organization Moves to Avoid Possible Con...,Noam Scheiber,In its latest effort to defuse a major public ...,0
18868,18868,BREAKING: Melania Trump’s Immigration Records ...,NaN,"on November 4, 2016 10:58 pm · \nGOP president...",1


In [0]:
# 新增Length欄位，紀錄每個標題文字總長度
# (Create a new column called Length that record every Headline length)
data['Length'] = [len(headline) for headline in data['title'].fillna('')]
data.head()

,id,title,author,text,label,Length
6392,6392,Politicians to Skip Boston St. Patrick’s Day P...,Liam Stack,Some of Massachusetts’s top politicians said o...,0,98
7160,7160,Turkish Government Arrests Opposition Parliame...,NaN,Turkish Government Arrests Opposition Parliame...,1,93
14155,14155,This Professor Devotes Her Life to Countering ...,Henry Wolff,"Loading Posted on November 2, 2016 This Profes...",1,96
833,833,Trump Organization Moves to Avoid Possible Con...,Noam Scheiber,In its latest effort to defuse a major public ...,0,85
18868,18868,BREAKING: Melania Trump’s Immigration Records ...,NaN,"on November 4, 2016 10:58 pm · \nGOP president...",1,133


In [0]:
detail = data['Length'].describe()
print(detail)

count    20800.000000
mean        72.325577
std         25.812558
min          0.000000
25%         59.000000
50%         75.000000
75%         87.000000
max        456.000000
Name: Length, dtype: float64


In [0]:
# Get all fake news
fliter = data['label'] == 1
pos = data[fliter]
print('是假新聞的數量(原始)：', len(pos))

# Get all true news
fliter = data['label'] == 0
neg = data[fliter]
print('真新聞的數量(原始)：', len(neg))

the_mean = min(len(pos), len(neg))
print("==============================================")

# 抓取所有 Label為1的 資料，並重新洗牌 (Random Shuffle)
p_data = pos.sample(n = the_mean)
print('取樣是假新聞資料總數：', len(p_data))

# 抓取所有 Label為0的 資料，並重新洗牌 (Random Shuffle)
n_data = neg.sample(n = the_mean)
print('取樣不是假新聞資料總數：', len(n_data))

是假新聞的數量(原始)： 10413
真新聞的數量(原始)： 10387
取樣是假新聞資料總數： 10387
取樣不是假新聞資料總數： 10387


In [0]:
test_split = 0.2
train_split = 1 - test_split



# 隨機抽樣80%的資料當訓練資料 而剩下的20%則當為測試資料
# (20% for Testing Data, others 80% for Training Data)
p_train_data = p_data.sample(frac = train_split)
p_test_data = p_data.drop(p_train_data.index)

n_train_data = n_data.sample(frac = train_split)
n_test_data = n_data.drop(n_train_data.index)

# 合併兩個類別的訓練資料與測試資料
# (Combined fake news and true news)
train_data = pd.concat([p_train_data, n_train_data])
test_data = pd.concat([p_test_data, n_test_data])

# 全部隨機洗牌 (Random Shuffle)
train_data = train_data.sample(frac = 1)
test_data = test_data.sample(frac = 1)

x_train_data = train_data['title'].fillna('')
y_train_data = train_data['label']
x_test_data = test_data['title'].fillna('')
y_test_data = test_data['label']

print('Train Data的Feature數量(已混合非假與假新聞)：', len(x_train_data))
print('Train Data的Label數量(已混合非假與假新聞)：', len(y_train_data))
print('Test Data的Feature數量(已混合非假與假新聞)：', len(x_test_data))
print('Test Data的Label數量(已混合非假與假新聞)：', len(y_test_data))


Train Data的Feature數量(已混合非假與假新聞)： 16620
Train Data的Label數量(已混合非假與假新聞)： 16620
Test Data的Feature數量(已混合非假與假新聞)： 4154
Test Data的Label數量(已混合非假與假新聞)： 4154


In [0]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
# token字典數量 (最常出現的4000字) (Create a token dictionary)
token_num = 4000 

# 擷取多少固定長度字數 (抓標題字數的平均值 72個長度)
# (Get a fix length, we chose the mean of the Headline length)
data_length = int(detail['mean'])

# 輸入向量維度 (Word Embeding output vector dimension)
output_length = 32 

dropout = 0.5
lstm_dim = 256


In [0]:
token = Tokenizer(num_words = token_num, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
token.fit_on_texts(x_train_data)

In [0]:
x_train_seq = token.texts_to_sequences(x_train_data)
x_test_seq = token.texts_to_sequences(x_test_data)


In [0]:
x_train = sequence.pad_sequences(x_train_seq, maxlen = data_length)
x_test = sequence.pad_sequences(x_test_seq, maxlen = data_length)

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional, TimeDistributed
from keras.callbacks import EarlyStopping

In [100]:
model = Sequential()
model.add(Embedding(output_dim = output_length, input_dim = token_num, input_length = data_length))
model.add(Dropout(dropout))

# using BLSTM (this will be better than LSTM, Avg acc is around 0.94
model.add(Bidirectional(LSTM(lstm_dim), merge_mode = 'sum'))
model.add(Dropout(dropout))

# using LSTM, Avg acc is around 0.93
#model.add(LSTM(lstm_dim))
#model.add(Dropout(dropout))

model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(dropout))

model.add(Dense(units = 1, activation = 'sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 72, 32)            128000    
_________________________________________________________________
dropout_38 (Dropout)         (None, 72, 32)            0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 256)               591872    
_________________________________________________________________
dropout_39 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_40 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 257       
Total para

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:
# 提早結束
es = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 2)

In [105]:
train_history = model.fit(x = x_train,
                         y = y_train_data,
                         validation_split = 0.4,
                         epochs = 1,
                         batch_size = 512,
                         verbose = 1,
                        
                         )


Train on 9972 samples, validate on 6648 samples
Epoch 1/1
9972/9972 [==============================] - 6s 584us/step - loss: 0.0539 - acc: 0.9807 - val_loss: 0.1156 - val_acc: 0.9639
